In [6]:
import urllib3
import certifi
import re
import mysql.connector as sql
from datetime import datetime
from bs4 import BeautifulSoup

#Urllib3
http = urllib3.PoolManager( cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

def generate_date(prev_date):
    from datetime import date, timedelta
    date_list = []
    for i in range(1, prev_date):
        tanggal = date.today() - timedelta(i)
        date_list.append(tanggal)
    
    return date_list

def insert(title, image, url, date):
    db_connection = sql.connect(host='127.0.0.1', database='news_aggregator', user='root', password='')
    cursor = db_connection.cursor()
    
    #Try Except
    try:
        data_table = ("INSERT INTO data (TITLE, IMAGE, URL, DATE) VALUES (%s, %s, %s, %s)")
        data_value = (title, image, url, date)

        cursor.execute(data_table, data_value)
        db_connection.commit()

    except sql.IntegrityError:
        print('duplicate entry')
        
    cursor.close()
    db_connection.close()

In [21]:
month = {'Januari': '01',
         'Februari': '02',
         'Maret': '03',
         'April': '04',
         'Mei': '05',
         'Juni': '06',
         'Juli': '07',
         'Agustus': '08',
         'September': '09',
         'Oktober': '10',
         'November': '11',
         'Desember': '12'
        }

In [28]:
def pull_data_techno(domain, pagination):
    
    for pag in range(1, pagination):
        if pag == 1:
            url = domain + '.html'
        else:
            url = domain + str(pag) + '.html'
        print(url)

        #Get article from website
        req = http.request('GET', url)
        soup = BeautifulSoup(req.data, 'lxml')

        try:
            containers = soup.find_all('ul', attrs={'class':'list-article-techno list-article-border list-unstyled'})
        except:
            print('container not found')
            break
        
        for cont in range(0, len(containers)):
            container = soup.find_all('ul', attrs={'class':'list-article-techno list-article-border list-unstyled'})[cont]
            
            try:
                boxes = container.find_all('li')
            except:
                print('box not found')
                break

            for i in range(0, len(boxes)):
                box = container.find_all('li')[i]

                #Get Title Article
                try:
                    title = box.find('p').text
                except:
                    print('h2.text not found')
                    break

                #Get Image
                try:
                    image = box.find('img').get('src')
                except:
                    print('image not found')
                    break

                #Get URL Article
                try:
                    url = box.find('a').get('href')
                except:
                    print('href not found')
                    break

                #Get Date
                try:
                    dateraw = box.find('span', attrs={'class': 'date'}).text
                    datestring = re.sub(r""" [0-9]{2}:[0-9]{2}""", '', dateraw)
                    datearr = datestring.split(' ')
                
                    for key in month:
                        if datearr[1] in key:
                            datearr[1] = month[key]

                    datestr = '/'.join(d for d in datearr)
                    date = datetime.strptime(datestr, '%d/%m/%Y')
                except:
                    print('date not found')
                    break
                    
                insert(title, image, url, date)

In [26]:
domain = [
    'https://www.techno.id/startup/index',
    'https://www.techno.id/gadget/index',
    'https://www.techno.id/apps/index',
    'https://www.techno.id/social/index',
    'https://www.techno.id/tech-news/index',
    'https://www.techno.id/photo/index',
    'https://www.techno.id/telco/index',
    'https://www.techno.id/science/index'
]

In [30]:
for url in domain:
    pull_data_techno(url, 2)

https://www.techno.id/startup/index.html
https://www.techno.id/gadget/index.html
https://www.techno.id/apps/index.html
https://www.techno.id/social/index.html
https://www.techno.id/tech-news/index.html
https://www.techno.id/photo/index.html
https://www.techno.id/telco/index.html
https://www.techno.id/science/index.html
